# Covid-19 Data Wrangling in Python

## Fatality statistics and ranks among countries

In [1]:
import pandas as pd
import numpy as np

print('pandas version: ', pd.__version__)
print('numpy version: ', np.version.version)

pandas version:  1.0.3
numpy version:  1.18.1


In [2]:
covid_daily_df = pd.read_csv('data/2020-04-03.csv')

covid_daily_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [3]:
covid_daily_df.dtypes

FIPS              float64
Admin2             object
Province_State     object
Country_Region     object
Last_Update        object
Lat               float64
Long_             float64
Confirmed           int64
Deaths              int64
Recovered           int64
Active              int64
Combined_Key       object
dtype: object

In [4]:
covid_daily_df.shape

(2625, 12)

In [5]:
# convert the FIPS County Code column to type of integer

covid_daily_df.FIPS = covid_daily_df.FIPS.fillna(-1) # get rid of NA 
covid_daily_df.FIPS = covid_daily_df.FIPS.astype(int)

covid_daily_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
1,22001,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
2,51001,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
3,16001,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
4,19001,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [6]:
# show all unique Country_Region codes

countries = covid_daily_df.Country_Region.unique()
print(countries)
len(countries)

['US' 'Canada' 'United Kingdom' 'China' 'Netherlands' 'Australia'
 'Denmark' 'France' 'Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Austria' 'Azerbaijan'
 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize'
 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil'
 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi' 'Cabo Verde'
 'Cambodia' 'Cameroon' 'Central African Republic' 'Chad' 'Chile'
 'Colombia' 'Congo (Brazzaville)' 'Congo (Kinshasa)' 'Costa Rica'
 "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus' 'Czechia' 'Diamond Princess'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Fiji' 'Finland' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Greece' 'Grenada' 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti'
 'Holy See' 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran'
 'Iraq' 'Ireland' 'Israel' 'It

181

In [7]:
# statistics grouped by countries
covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum})

,Confirmed
Country_Region,
Afghanistan,281
Albania,304
Algeria,1171
Andorra,439
Angola,8
...,...
Venezuela,153
Vietnam,237
West Bank and Gaza,194


In [8]:
# construct a dataframe with five columns: 
# Country_Region (index), Confirmed, Deaths, Recovered, Active
df_country_cases = covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum, 'Deaths': np.sum, 'Recovered': np.sum, 'Active': np.sum})

df_country_cases

,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,281,6,10,265
Albania,304,17,89,198
Algeria,1171,105,62,1004
Andorra,439,16,16,407
Angola,8,2,1,5
...,...,...,...,...
Venezuela,153,7,52,94
Vietnam,237,0,85,152
West Bank and Gaza,194,1,21,172


In [9]:
# show counties that do not have any corona patients yet
len(df_country_cases.loc[df_country_cases['Confirmed'] == 0])

0

In [10]:
# last operation tells that all countries in this csv file have confirmed cases. 
# now we can calculate the fatality rate

df_country_cases['fatality_rate'] = df_country_cases.Deaths / df_country_cases.Confirmed

fatalities = df_country_cases.query('(Deaths > 0) & (Confirmed > 30)') \
                            .sort_values(by = 'fatality_rate', ascending = False)

fatalities[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
Italy,119827,14681,19758,85388,0.122518
San Marino,245,30,21,194,0.122449
France,65202,6520,14135,44547,0.099997
Bangladesh,61,6,26,29,0.098361
Congo (Kinshasa),134,13,3,118,0.097015
Netherlands,15821,1490,260,14071,0.094179
Spain,119199,11198,30513,77488,0.093944
United Kingdom,38689,3611,208,34870,0.093334
Indonesia,1986,181,134,1671,0.091138


In [11]:
fatalities_asc = df_country_cases.query('(Deaths > 0) & (Confirmed > 30)').sort_values(by = 'fatality_rate', ascending = True)

fatalities_asc[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
New Zealand,868,1,103,764,0.001152
Latvia,493,1,1,491,0.002028
Slovakia,450,1,10,439,0.002222
Qatar,1075,3,93,979,0.002791
Iceland,1364,4,309,1051,0.002933
Oman,252,1,57,194,0.003968
Singapore,1114,5,282,827,0.004488
Cote d'Ivoire,218,1,19,198,0.004587
Costa Rica,416,2,11,403,0.004808


In [12]:
df_country_cases.Deaths.sum()

58787

In [13]:
# fatalitiy over all countries having more than 30 confirmed cases. 

fatality_partial = fatalities.Deaths.sum() / fatalities.Confirmed.sum()

fatality_partial

0.05372345872250788

In [14]:
# fatalitiy over all countries, regardless of how many confirmed cases and deaths. 

fatality_overall = df_country_cases.Deaths.sum() / df_country_cases.Confirmed.sum()

fatality_overall

0.05364183601495369

In [15]:
# define a function which returns the rank of the fatality

def fatality_rank( country: str, asc: bool ) -> int:
    indices = fatalities.sort_values(by = 'fatality_rate', ascending = asc).index
    return indices.get_loc(country)


In [16]:
ranks_asc = fatalities.sort_values(by = 'fatality_rate', ascending = True).index

def get_fatality_and_rank(country: str) -> pd.Series:
    fatality = fatalities.loc[country].fatality_rate

    rank_asc = ranks_asc.get_loc(country) + 1
    rank_dsc = len(ranks_asc) - rank_asc + 1

    return pd.Series({'Country_Region': country, 'fatality_rate': fatality, 'rank_asc': rank_asc, 'rank_dsc': rank_dsc})


In [17]:
# test the function get_fatality_and_rank

get_fatality_and_rank('Italy')

Country_Region       Italy
fatality_rate     0.122518
rank_asc               118
rank_dsc                 1
dtype: object

In [18]:
# how do these countries score in the fatality_rate?

cols = ['Country_Region', 'fatality_rate', 'rank_asc', 'rank_dsc']
countries = ['Austria', 'China', 'Germany', 'Italy', 'New Zealand', 'San Marino', 'Singapore', 'US']

df_fatality_scores = pd.DataFrame(columns = cols)

for country in countries:
    df_fatality_scores = df_fatality_scores.append(get_fatality_and_rank(country), ignore_index=True)

df_fatality_scores

,Country_Region,fatality_rate,rank_asc,rank_dsc
0,Austria,0.014578,42,77
1,China,0.040310,86,33
2,Germany,0.013987,39,80
3,Italy,0.122518,118,1
4,New Zealand,0.001152,1,118
5,San Marino,0.122449,117,2
6,Singapore,0.004488,7,112
7,US,0.025716,65,54


In [19]:
# basic statistics of the fatality
fatalities.fatality_rate.describe()

count    118.000000
mean       0.032025
std        0.028041
min        0.001152
25%        0.011588
50%        0.024637
75%        0.041772
max        0.122518
Name: fatality_rate, dtype: float64

In [20]:
# median and mode of fatality rate

fatality_median = fatalities.fatality_rate.median()
fatality_mode = fatalities.fatality_rate.mode()

print("Fatality median is {} and mode is {}".format(fatality_median, fatality_mode))


Fatality median is 0.024636958892348383 and mode is 0    0.012931
dtype: float64


In [21]:
# the mode above does not make sense, let's fix it.

# first, segment and sort fatality values into bins
bins = pd.cut(fatalities['fatality_rate'], 10, include_lowest=True)

bins

# I'd like to see the content of each bin, what to do?

Country_Region
Italy                                 (0.11, 0.123]
San Marino                            (0.11, 0.123]
France                               (0.0982, 0.11]
Bangladesh                           (0.0982, 0.11]
Congo (Kinshasa)                   (0.0861, 0.0982]
                                 ...               
Iceland             (3.000000000000008e-05, 0.0133]
Qatar               (3.000000000000008e-05, 0.0133]
Slovakia            (3.000000000000008e-05, 0.0133]
Latvia              (3.000000000000008e-05, 0.0133]
New Zealand         (3.000000000000008e-05, 0.0133]
Name: fatality_rate, Length: 118, dtype: category
Categories (10, interval[float64]): [(3.000000000000008e-05, 0.0133] < (0.0133, 0.0254] < (0.0254, 0.0376] < (0.0376, 0.0497] ... (0.074, 0.0861] < (0.0861, 0.0982] < (0.0982, 0.11] < (0.11, 0.123]]

## Joined with Population data from United Nations 

Poplation is in thousand persons.  

In [22]:
population = pd.read_csv('data/WPP2019_TotalPopulationBySex.csv')
population.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486


In [23]:
# we only need the data of year 2020
pop2019 = population[population.Time == 2019]
pop2019.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
69,4,Afghanistan,2,Medium,2019,2019.5,19529.727,18512.030,38041.757,58.269
953,903,Africa,2,Medium,2019,2019.5,653513.680,654550.496,1308064.176,44.119
1837,1823,African Group,2,Medium,2019,2019.5,652644.714,653675.858,1306320.572,44.464
1988,1560,African Union,2,Medium,2019,2019.5,652949.469,653953.561,1306903.030,44.085
2139,2080,African Union: Central Africa,2,Medium,2019,2019.5,76945.498,77068.207,154013.705,29.192


In [24]:
len(pop2019)

477

### Now we need to unify the lation names of the population data and the corona data 

In [25]:
# The set of location names of the population data is much bigger than corona's. 
# So we first find out which country names are not in the population data

countryNames_covid19_only = df_country_cases[df_country_cases.index.isin(pop2019.Location) == False]

countryNames_covid19_only.index

Index(['Bolivia', 'Brunei', 'Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Cote d'Ivoire', 'Diamond Princess', 'Iran', 'Korea, South', 'Kosovo',
       'Laos', 'MS Zaandam', 'Moldova', 'Russia', 'Syria', 'Taiwan*',
       'Tanzania', 'US', 'Venezuela', 'Vietnam', 'West Bank and Gaza'],
      dtype='object', name='Country_Region')

In [26]:
len(countryNames_covid19_only)

21

In [27]:
pop2019[pop2019.Location.str.contains('Pales')]

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
236754,275,State of Palestine,2,Medium,2019,2019.5,2526.35,2455.072,4981.422,827.479


In [28]:
pop2019.Location.replace(
    {'Bolivia (Plurinational State of)':'Bolivia', 
    'Brunei Darussalam':'Brunei', 
    'Myanmar': 'Burma', 
    'Congo':'Congo (Brazzaville)', 
    'Democratic Republic of the Congo':'Congo (Kinshasa)', 
    'Côte d\'Ivoire':'Cote d\'Ivoire', 
    'Iran (Islamic Republic of)':'Iran', 
    'Republic of Korea':'Korea, South', 
    'Lao People\'s Democratic Republic':'Laos', 
    'Republic of Moldova':'Moldova', 
    'Russian Federation':'Russia', 
    'Syrian Arab Republic':'Syria', 
    'China, Taiwan Province of China':'Taiwan*', 
    'United Republic of Tanzania':'Tanzania', 
    'United States of America':'US', 
    'Venezuela (Bolivarian Republic of)':'Venezuela', 
    'Viet Nam':'Vietnam', 
    'State of Palestine':'West Bank and Gaza'}, inplace=True)

In [29]:
# drop unnecessary columns

df_pop2019 = pop2019.drop(columns=['LocID','VarID','Variant','Time','MidPeriod'])
df_pop2019.head()

,Location,PopMale,PopFemale,PopTotal,PopDensity
69,Afghanistan,19529.727,18512.030,38041.757,58.269
953,Africa,653513.680,654550.496,1308064.176,44.119
1837,African Group,652644.714,653675.858,1306320.572,44.464
1988,African Union,652949.469,653953.561,1306903.030,44.085
2139,African Union: Central Africa,76945.498,77068.207,154013.705,29.192


In [30]:
# now join the corona and population data frames

df_covid19_pop2019 = pd.merge(df_country_cases, df_pop2019, how='inner', left_on='Country_Region', right_on='Location')
df_covid19_pop2019.head()

,Confirmed,Deaths,Recovered,Active,fatality_rate,Location,PopMale,PopFemale,PopTotal,PopDensity
0,281,6,10,265,0.021352,Afghanistan,19529.727,18512.030,38041.757,58.269
1,304,17,89,198,0.055921,Albania,1466.785,1414.128,2880.913,105.143
2,1171,105,62,1004,0.089667,Algeria,21749.666,21303.388,43053.054,18.076
3,439,16,16,407,0.036446,Andorra,NaN,NaN,77.146,164.140
4,8,2,1,5,0.250000,Angola,15744.779,16080.520,31825.299,25.528


In [31]:
df_covid19_pop2019.shape

(178, 10)

In [32]:
# correlation between fatality_rate and PopDensity??

corr_fatality_popDensity = df_covid19_pop2019.fatality_rate.corr(df_covid19_pop2019.PopDensity)
corr_fatality_popDensity

-0.05427594982763866

In [33]:
corr_confirmed_popTotal = df_covid19_pop2019.Confirmed.corr(df_covid19_pop2019.PopTotal)
corr_confirmed_popTotal

0.28965686170935384

In [34]:
corr_deaths_popTotal = df_covid19_pop2019.Deaths.corr(df_covid19_pop2019.PopTotal)
corr_deaths_popTotal

0.17186799844692763